In [23]:
import os
import json
import pandas as pd
from tqdm import tqdm


In [25]:
def flatten_json(obj, prefix=''):
    flat = {}
    if isinstance(obj, dict):
        for k, v in obj.items():
            full_key = f"{prefix}.{k}" if prefix else k
            flat.update(flatten_json(v, full_key))
    elif isinstance(obj, list):
        collected = {}
        for item in obj:
            item_flat = flatten_json(item, prefix)
            for k, v in item_flat.items():
                if k not in collected:
                    collected[k] = []
                collected[k].append(v)
        for k, vlist in collected.items():
            flat[k] = '; '.join(map(str, vlist))
    else:
        flat[prefix] = obj
    return flat


In [27]:
def load_jsons_in_batches(base_dir, batch_size=10000):
    all_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.json'):
                all_files.append(os.path.join(root, file))
    
    all_dfs = []
    for i in tqdm(range(0, len(all_files), batch_size), desc="Processing CVEs"):
        batch_files = all_files[i:i+batch_size]
        batch_records = []
        for path in batch_files:
            try:
                with open(path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    flat = flatten_json(data)
                    flat['cve_json_file'] = os.path.relpath(path, base_dir)
                    batch_records.append(flat)
            except Exception as e:
                print(f"Error reading {path}: {e}")
        batch_df = pd.DataFrame(batch_records)
        all_dfs.append(batch_df)
    
    final_df = pd.concat(all_dfs, ignore_index=True)
    return final_df


In [29]:
base_directory = 'C:/Users/majit/Downloads/cvelistV5-main'

df_flat = load_jsons_in_batches(base_dir=base_directory, batch_size=10000)


Processing CVEs: 100%|██████████| 30/30 [03:31<00:00,  7.06s/it]


In [31]:
df_flat.head()

,type,cve_json_file,fetchTime,numberOfChanges,new.cveId,new.cveOrgLink,new.githubLink,new.dateUpdated,updated.cveId,updated.cveOrgLink,...,containers.cna.x_legacyV4Record.impact.cvss.vulnerabilityResponseEffort,containers.cna.x_legacyV4Record.CNA_private.Current-Status,containers.cna.x_legacyV4Record.CNA_private.affectedKeywords,containers.cna.x_affectedList,containers.cna.metrics.other.content.selections.namespace,containers.cna.metrics.other.content.selections.version,containers.cna.metrics.other.content.selections.values,containers.cna.metrics.other.content.selections.name,containers.cna.metrics.other.content.schemaVersion,containers.cna.cpeApplicability.operator
0,module,.github\workflows\dist\package.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,cves\delta.json,2025-06-23T21:36:20.453Z,1,CVE-2025-6525,https://www.cve.org/CVERecord?id=CVE-2025-6525,https://raw.githubusercontent.com/CVEProject/c...,2025-06-23T21:31:05.407Z,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,cves\deltaLog.json,2025-06-23T21:36:20.453Z; 2025-06-23T21:07:03....,1; 2; 9; 10; 9; 7; 12; 3; 2; 6; 53; 5; 3; 1; 8...,CVE-2025-6525; CVE-2025-52561; CVE-2025-6524; ...,https://www.cve.org/CVERecord?id=CVE-2025-6525...,https://raw.githubusercontent.com/CVEProject/c...,2025-06-23T21:31:05.407Z; 2025-06-23T21:00:13....,CVE-2025-49968; CVE-2025-49969; CVE-2025-49970...,https://www.cve.org/CVERecord?id=CVE-2025-4996...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,cves\1999\0xxx\CVE-1999-0001.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,cves\1999\0xxx\CVE-1999-0002.json,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
df_flat.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299126 entries, 0 to 299125
Data columns (total 524 columns):
 #    Column                                                                                                                  Dtype  
---   ------                                                                                                                  -----  
 0    type                                                                                                                    object 
 1    cve_json_file                                                                                                           object 
 2    fetchTime                                                                                                               object 
 3    numberOfChanges                                                                                                         object 
 4    new.cveId                                                                     

In [47]:
df_flat.to_csv('C:/Users/majit/Downloads/dflattened_cve_dataset.csv', index=False)


In [45]:
non_null_counts = df_flat.notnull().sum().reset_index()
non_null_counts.columns = ['column', 'non_null_count']
non_null_counts = non_null_counts.sort_values(by='non_null_count', ascending=False)
non_null_counts.head(10)  


,column,non_null_count
1,cve_json_file,299126
56,cveMetadata.state,299123
52,cveMetadata.cveId,299123
57,dataType,299123
50,cveMetadata.assignerOrgId,299123
58,dataVersion,299123
22,containers.cna.providerMetadata.orgId,299123
54,cveMetadata.dateReserved,299123
55,cveMetadata.dateUpdated,299123
21,containers.cna.providerMetadata.dateUpdated,299121


In [51]:
non_null_counts.to_excel('C:/Users/majit/Downloads/non_null_cols.xlsx', index=False)

In [55]:
types_series = df_flat['containers.cna.problemTypes.descriptions.type'].dropna()

distinct_types = set(
    t.strip()
    for entry in types_series
    for t in str(entry).split(';')
)


distinct_types = sorted(distinct_types)

print("Distinct problemTypes.descriptions.type values:")
for t in distinct_types:
    print("-", t)

Distinct problemTypes.descriptions.type values:
- BASM (Broken Authentication & Session Management)
- CWE
- DoS (Denial of Service)
- File Inclusion
- Impact
- Improper Authorization
- Information Disclosure
- Injection
- Open Redirect
- Other
- PrivEsc (Privilege Escalation)
- RCE (Remote Code Execution)
- Reflected XSS
- Security Misconfiguration
- Stored XSS
- cwe
- problem_type
- text
